In [1]:
import numpy as np
import numpy.linalg as lg 
from scipy.sparse.linalg import gmres
from tabulate import tabulate
from time import time
from  scipy.io import mmread
A1=np.array(mmread('/content/bcsstk20.mtx.gz').todense())
# A1.shape=(485,485) real symmetric positive definite
A2=np.array(mmread('/content/mcca.mtx.gz').todense())
# A2.shape=(180,180) real unsymmetric  
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A4.shape=(817,817) real symmetric positive definite

C1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# C1.shape=(1806, 1806) real symmetric positive definite
C2=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# C2.shape=(3562,3562) real symmetric positive definite
C3=np.array(mmread('/content/bcsstk16.mtx.gz').todense())

B1=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B2=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
B3=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
B4=np.array(mmread('/content/bcsstm26.mtx.gz').todense())


D1=np.array(mmread('/content/impcol_e.mtx.gz').todense())
D2=np.array(mmread('/content/impcol_a.mtx.gz').todense())
D3=np.array(mmread('/content/impcol_b.mtx.gz').todense())
D4=np.array(mmread('/content/impcol_c.mtx.gz').todense())
D5=np.array(mmread('/content/impcol_d.mtx.gz').todense())


E1=np.array(mmread('/content/west0067.mtx.gz').todense())
E2=np.array(mmread('/content/jpwh_991.mtx.gz').todense())
E3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
E4=np.array(mmread('/content/impcol_e.mtx.gz').todense())
E5=np.array(mmread('/content/sherman4.mtx.gz').todense())
E6=np.array(mmread('/content/orsreg_1.mtx.gz').todense())
E7=np.array(mmread('/content/orsirr_1.mtx.gz').todense())
E8=np.array(mmread('/content/add20.mtx.gz').todense())
E9=np.array(mmread('/content/gre__185.mtx.gz').todense())
E10=np.array(mmread('/content/fs_680_3.mtx.gz').todense())
E11=np.array(mmread('/content/fs_680_2.mtx.gz').todense())
E12=np.array(mmread('/content/west0381.mtx.gz').todense())
E13=np.array(mmread('/content/e05r0200.mtx.gz').todense())




# **The GMRES algorithm**

In [2]:
def gmres_iter(A, b,x0,maxiter,restart,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return gmres(A, b, callback=callback,tol = tol,restart=restart,maxiter=maxiter),num_iters

def The_GMRES_algorithm(A, b, x, m, tol, itermax):
    r      = b-np.dot(A, x)
    beta   = lg.norm(r)
    ris    = np.zeros(itermax)
    V      = np.zeros((len(b), m+1))
    V[:,0] = r/beta
    H      = np.zeros((m+1,m))
    k      = 0
    while  k < itermax and beta>tol:
        ris[k]   = beta
        for j in range(m):
            w = np.dot(A,V[:, j])
            for i in range(j+1):            
                H[i,j]= sum(w*V[:,i])
                w     = w - H[i,j]*V[ :,i]
            H[j+1,j] = lg.norm(w)
            if H[j+1,j]==0:
                break
            V[:, j+1] = w / H[j+1,j] 
        e1      = np.zeros(m+1)
        e1[0]   = 1
        Vm      = V[:,:m]                                         # Vm obtenue en supprimant la dernière colonne de V
        Q, R    = lg.qr(H,mode='complete')                        # application du QR à la matrice H de taille (m+1)*m
        Rm      = R[:m,:m]                                        # matrice extraite en supprimant la dernière colonne de R
        g       = beta*np.dot(np.transpose(Q),e1)                 # vecteur de taille m+1
        gm      = g[:m]                                           # vecteur de taille m (en garde les m premières composantes)
        ym      = lg.solve(Rm, gm)
        x       = x + np.dot(Vm, ym)
        r       = b - np.dot(A, x)
        beta    = lg.norm(r)
        V[:, 0] = r/beta
        k       = k + 1
    return x, k


# Test:

In [4]:
N=100
m=restart=[20,30,N]
A = 2*np.eye(N) -np.diag(np.ones(N-1),-1)-np.diag(np.ones(N-1),1)
b = np.ones(N)
tol=1e-5
itermax=1000
print('codition umber :',lg.cond(A))

codition umber : 4133.642926801328


In [7]:
my_data = []
for i in range(len(m)):
  l = []
  b=np.ones(N)
  x = np.zeros(N)
  itersc = gmres_iter(A,b,x,itermax,restart[i],tol)[1] 
  t3 = time()
  y=gmres(A,b,x,tol,m[i],itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(A,y)-b)
  t1 = time()
  x,iterm= The_GMRES_algorithm(A, b, x, m[i], tol, itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(A,x)-b)
  l.append(m[i])
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['m','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid"))

+-----+----------------+---------+-------------+------------------+-----------+-------------+
|   m |   Our_nbr_iter |    time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+=====+================+=========+=============+==================+===========+=============+
|  20 |             74 | 957.854 | 8.48012e-06 |             1000 |   53.6418 | 9.98424e-05 |
+-----+----------------+---------+-------------+------------------+-----------+-------------+
|  30 |             32 | 853.234 | 8.81485e-06 |              795 |   37.1377 | 9.94065e-05 |
+-----+----------------+---------+-------------+------------------+-----------+-------------+
| 100 |              1 | 264.627 | 3.69937e-11 |               50 |    2.5239 | 5.54502e-12 |
+-----+----------------+---------+-------------+------------------+-----------+-------------+


# **Cluster 1:**

In [12]:
matricies=[B1,E2,B2,B3,B4,A3,A4,E5,E6,E7,E8]
tol=1e-5
itermax = 10000
m=restart=30

In [13]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  t3 = time()
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)

# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']   
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------+-------------+------------------+-------------+-------------+
| matricies    |   condition number |   Our_nbr_iter |       time |         res |   scipy_nbr_iter |     sc_time |      sc_res |
+==============+====================+================+============+=============+==================+=============+=============+
| (3600, 3600) |            23.7154 |              1 |    941.777 | 1.0025e-12  |                3 |    41.3268  | 2.87905e-13 |
+--------------+--------------------+----------------+------------+-------------+------------------+-------------+-------------+
| (991, 991)   |           142.045  |              2 |    385.608 | 2.03457e-07 |               37 |    16.4621  | 0.000230357 |
+--------------+--------------------+----------------+------------+-------------+------------------+-------------+-------------+
| (138, 138)   |           941.295  |              4 |    132.198 | 7.92443e-09 |               7

# **Cluster 2:**

In [26]:
matricies=[D1,D2,C3,C1,A1,C2]
tol=1e-5
itermax = 1000
m=restart=30

In [27]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  t3 = time()
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']   
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------------+----------+------------------+-----------+----------+
| matricies    |   condition number |   Our_nbr_iter |             time |      res |   scipy_nbr_iter |   sc_time |   sc_res |
+==============+====================+================+==================+==========+==================+===========+==========+
| (225, 225)   |        7.10097e+06 |           1000 |  49479.9         | 13.5361  |             1000 |   1742.63 | 13.5361  |
+--------------+--------------------+----------------+------------------+----------+------------------+-----------+----------+
| (207, 207)   |        1.35164e+08 |           1000 |  45958.3         | 13.4093  |             1000 |   1633.87 | 13.4093  |
+--------------+--------------------+----------------+------------------+----------+------------------+-----------+----------+
| (4884, 4884) |        4.94318e+09 |           1000 |      1.38033e+06 |  8.59811 |             1000 | 486985 

# **Cluster 3:**

In [19]:
matricies=[E2,E5,E6,E7,E8]
tol=1e-5
itermax = 1000
m=restart=30

In [20]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  t3 = time()
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']   
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |      time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+===========+=============+==================+===========+=============+
| (991, 991)   |            142.045 |              2 |   392.645 | 2.03457e-07 |               37 |   17.4658 | 0.000230357 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (1104, 1104) |           2178.63  |             16 |  3609.78  | 6.26214e-06 |              333 |  164.054  | 0.000326244 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (2205, 2205) |           6745.27  |             10 |  5005.97  | 6.76815e-06 |              147 |  420.62   | 0.0004

**Size of Krylov subspace: 20**


In [22]:
matricies=[E2,E5,E6,E7,E8]
tol=1e-5
itermax = 1000
m=restart=20

In [23]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = gmres_iter(matricies[i],b,x,itermax,restart,tol)[1] 
  t3 = time()
  y=gmres(matricies[i],b,x,tol,m,itermax)[0]
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= The_GMRES_algorithm(matricies[i], b, x, m, tol, itermax)
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms 
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']   
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |      time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+===========+=============+==================+===========+=============+
| (991, 991)   |            142.045 |              3 |   295.406 | 3.19246e-06 |               46 |   20.9105 | 0.000244903 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (1104, 1104) |           2178.63  |             31 |  3324.01  | 8.03502e-06 |              478 |  245.253  | 0.000330533 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (2205, 2205) |           6745.27  |             19 |  4729.16  | 8.22653e-06 |              189 |  474.733  | 0.0004